## Initialization

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

## Importing the Data

In [3]:
dataset = pd.read_csv('data_banknote_authentication.txt', header = None)
dataset.shape

(1372, 5)

In [4]:
dataset.head()

,0,1,2,3,4
0,3.62160,8.6661,-2.8073,-0.44699,0
1,4.54590,8.1674,-2.4586,-1.46210,0
2,3.86600,-2.6383,1.9242,0.10645,0
3,3.45660,9.5228,-4.0112,-3.59440,0
4,0.32924,-4.4552,4.5718,-0.98880,0


## Create X and Y

In [5]:
X = dataset.iloc[:, 0:4].values
Y = dataset.iloc[:, 4].values

In [6]:
X

array([[  3.6216 ,   8.6661 ,  -2.8073 ,  -0.44699],
       [  4.5459 ,   8.1674 ,  -2.4586 ,  -1.4621 ],
       [  3.866  ,  -2.6383 ,   1.9242 ,   0.10645],
       ..., 
       [ -3.7503 , -13.4586 ,  17.5932 ,  -2.7771 ],
       [ -3.5637 ,  -8.3827 ,  12.393  ,  -1.2823 ],
       [ -2.5419 ,  -0.65804,   2.6842 ,   1.1952 ]])

In [7]:
Y

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

## Preprocess the Data

In [8]:
Y = Y.reshape(len(Y), 1)
Y

array([[0],
       [0],
       [0],
       ..., 
       [1],
       [1],
       [1]], dtype=int64)

In [9]:
ohe_Y = OneHotEncoder(categorical_features = [0])

In [10]:
Y = ohe_Y.fit_transform(Y).toarray()
Y

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

## Create Train and Test Data

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = 4)

In [12]:
X_train.shape

(1029, 4)

In [13]:
Y_train.shape

(1029, 2)

In [14]:
X_test.shape

(343, 4)

In [15]:
Y_test.shape

(343, 2)

## Create TF Classifier

In [16]:
num_features = X.shape[1]
num_features

4

In [17]:
num_classes = Y.shape[1]
num_classes

2

In [18]:
# Y = W1.X1 + W2.X2 + ... + Wn.Xn + B
# output = 1 / (1 + e ^ (- Y))
# Create Weights and Biases Variables

W = tf.Variable(tf.zeros([num_features, num_classes]))
B = tf.Variable(tf.zeros([num_classes]))

In [19]:
# Create x and y_ placeholders to hold actual data
x = tf.placeholder(tf.float32, [None, num_features])
y_ = tf.placeholder(tf.float32, [None, num_classes])

In [20]:
# Calculate y which holds the model's predicted values
Wx = tf.matmul(x, W)
y = Wx + B

In [21]:
# Create a cost function to minimize
cost_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits = y))

In [22]:
# Create an Optimizer to minimize the cost
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(cost_cross_entropy)

In [23]:
def trainTheData(num_steps, optimizer_to_use, batch_size):
    init = tf.global_variables_initializer()
    # Initialize all the global variables
    
    with tf.Session() as sess:
        sess.run(init)
        
        for i in range(num_steps):
            
            # Calculate the batch start index
            if batch_size == len(X_train):
                batch_start_index = 0
            elif batch_size > len(X_train):
                raise ValueError("Batch Size : " + str(batch_size) + " cannot be greater than Data Size : ", len(X_train))
            else:
                batch_start_index = (i * batch_size) % (len(X_train) - batch_size)
            
            # Calculate the batch end index
            batch_end_index = batch_size + batch_start_index
            
            # Create batch X and Y values
            batch_X_values = X_train[batch_start_index : batch_end_index]
            batch_Y_values = Y_train[batch_start_index : batch_end_index]
            
            # Create feed dict to feed the optimizer
            feed = {x : np.array(batch_X_values), y_ : np.array(batch_Y_values)}
            
            sess.run(optimizer_to_use, feed_dict = feed)
            
            if (i + 1) % 50 == 0:
                print("After " + str(i + 1) + " iterations, cross_entropy : ", sess.run(cost_cross_entropy, feed_dict = feed))
                print("W : ", sess.run(W))
                print("B : ", sess.run(B))
        
        # Calculate the predicted values for Test Data
        Y_pred = sess.run(y, feed_dict = {x : np.array(X_test)})
        
        # Convert the logits into tensors
        Y_pred_tensors = tf.convert_to_tensor(np.array(Y_pred))
        
        # Apply softmax function to the tensors of logits
        apply_softmax = tf.nn.softmax(Y_pred_tensors)
        
        # Calculate the class to which it belongs
        Y_pred = np.argmax(sess.run(apply_softmax), axis = 1)
        
        sess.close()
    return Y_pred

In [24]:
Y_pred_classes = trainTheData(num_steps = 500, optimizer_to_use = optimizer, batch_size = len(X_train))

After 50 iterations, cross_entropy :  0.0497393
W :  [[ 0.93982112 -0.93982112]
 [ 0.54547602 -0.54547596]
 [ 0.62691528 -0.62691534]
 [ 0.15182392 -0.15182397]]
B :  [-0.82157236  0.82157254]
After 100 iterations, cross_entropy :  0.038668
W :  [[ 1.08588433 -1.08588421]
 [ 0.6247645  -0.62476426]
 [ 0.74108493 -0.74108523]
 [ 0.10866363 -0.10866367]]
B :  [-1.12573004  1.1257304 ]
After 150 iterations, cross_entropy :  0.0343103
W :  [[ 1.20006168 -1.20006156]
 [ 0.68346691 -0.68346661]
 [ 0.82289255 -0.82289284]
 [ 0.08949518 -0.08949527]]
B :  [-1.30042565  1.30042601]
After 200 iterations, cross_entropy :  0.031887
W :  [[ 1.2945708  -1.29457068]
 [ 0.73191404 -0.73191363]
 [ 0.88879639 -0.88879681]
 [ 0.08097432 -0.08097441]]
B :  [-1.4208802   1.42088044]
After 250 iterations, cross_entropy :  0.0302975
W :  [[ 1.37574267 -1.37574255]
 [ 0.77357823 -0.77357781]
 [ 0.94466412 -0.94466454]
 [ 0.07768676 -0.07768682]]
B :  [-1.51247013  1.51247036]
After 300 iterations, cross_entro

In [25]:
Y_pred_classes

array([0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       0, 1,

## Check the accuracy

In [26]:
Y_test_classes = np.argmax(Y_test, axis = 1)

In [27]:
accuracy_score(Y_test_classes, Y_pred_classes)

0.99708454810495628

In [28]:
confusion_matrix(Y_test_classes, Y_pred_classes)

array([[195,   1],
       [  0, 147]], dtype=int64)